## Create multiple SHOP yaml files with different price in the first 24 h based on a "base file" 

In [ ]:
import yaml
from datetime import datetime, timedelta
from copy import deepcopy

with open('../tests/data/demo/v1/files/fornebu_model_with_time_series_values_base.yaml', 'r') as f:
    base_case_dict = yaml.safe_load(f)
datetimes = [datetime(2023, 7, 7, 22, 0) + timedelta(hours=hh) for hh in range(24)]
def add_to_items(ts_dict, item_keys, add_value):
    for key in item_keys:
        if key in ts_dict:
            ts_dict[key] += add_value

add_to_price = range(-400, 600, 10)
model_file_names = []
for add in add_to_price:
    new_case_dict = deepcopy(base_case_dict)
    add_to_items(new_case_dict['model']['market']["Dayahead"]["buy_price"], datetimes, add)
    add_to_items(new_case_dict['model']['market']["Dayahead"]["sale_price"], datetimes, add)
    if add < 0:
        suffix = f"minus_{abs(add)}"
    elif add>0:
        suffix = f"plus_{add}"
    else:
        continue
    file_name = f'fornebu_model_with_time_series_values_price_{suffix}.yaml'
    model_file_names.append(file_name)
    with open(f'../tests/data/demo/v1/files/{file_name}', 'w') as new_file:
        yaml.safe_dump(new_case_dict, new_file, sort_keys=False)
    #print(f"add: {add}")
    #print(f"buy_price: {new_case_dict['model']['market']['Dayahead']['buy_price']}")


## Add files to resync_configuration (to make sure they are uploaded)

In [ ]:
resync_config_file_path = '../tests/data/demo/v1/resync_configuration.yaml'
with open(resync_config_file_path, 'r') as f:
    resync_configuration_dict = yaml.safe_load(f)
files_config_metadata = resync_configuration_dict["file_configuration"]["file_metadata"]
file_names_in_config = [file["name"] for file in files_config_metadata if file["directory"] == "/files"]
for file_name in model_file_names:
    if file_name not in file_names_in_config:
        files_config_metadata.append({
            "name": file_name,
            "external_id": file_name.split(".")[0],
            "directory": "/files",
            "source": "resync",
            "mime_type": "application/yaml"})
with open(resync_config_file_path, 'w') as f:
    yaml.safe_dump(resync_configuration_dict, f, sort_keys=False)

## Define ShopModel and ShopScenario instances and update ShopScenarioSet

In [ ]:
instances_def_path = "../tests/data/demo/v1/fornebu_no_time_series"

def underscore_to_space(s):
    return s.replace("_", " ")

shop_model_file = f'{instances_def_path}/shop_model.yaml'
with open(shop_model_file, 'r') as f:
    shop_model_dict_list = yaml.safe_load(f)
shop_model_names_in_file = [m["name"] for m in shop_model_dict_list]

shop_scenario_file = f'{instances_def_path}/shop_scenario.yaml'
with open(shop_scenario_file, 'r') as f:
    shop_scenario_dict_list = yaml.safe_load(f)
shop_scenario_names_in_file = [m["name"] for m in shop_scenario_dict_list]

shop_scenario_set_file = f'{instances_def_path}/shop_scenario_set.yaml'
with open(shop_scenario_set_file, 'r') as f:
    shop_scenario_set_dict_list = yaml.safe_load(f)
for scenario_set in shop_scenario_set_dict_list:
    if scenario_set["name"] == "100 scenarios no time series":
        shop_scenario_set_dict = scenario_set
        break
shop_scenario_set_scenarios = shop_scenario_set_dict["scenarios"]

for add_price in add_to_price:
    if add_price < 0:
        suffix = f"minus_{abs(add_price)}"
    elif add_price > 0:
        suffix = f"plus_{add_price}"
    else:
        continue
    shop_model_name = f"Fornebu with time series values price {underscore_to_space(suffix)}"
    if shop_model_name not in shop_model_names_in_file:
        shop_model_dict_list.append({
            'name': shop_model_name,
            'model_version': '1',
            'shop_version': '15.4.1.1',
            'penalty_limit': 10000.0,
            'model': f"[external_id]fornebu_model_with_time_series_values_price_{suffix}",
            'cog_shop_files_config': [],
            'base_attribute_mappings': []
        })
    
    shop_scenario_name=f"Fornebu no time series price {underscore_to_space(suffix)}"
    if shop_scenario_name not in shop_scenario_names_in_file:
        shop_scenario_dict_list.append({
            "name": shop_scenario_name,
            "model": f"[name]{shop_model_name}",
            "commands": "[name|type:ShopCommands]default",
            "source": "resync",
            "attribute_mappings_override": [],
            "output_definition": ["[name]market price", "[name]plant production", "[name]plant consumption"],
        })

    if f"[name]{shop_scenario_name}" not in shop_scenario_set_scenarios:
        shop_scenario_set_scenarios.append(f"[name]{shop_scenario_name}")

with open(shop_model_file, 'w') as f:
    yaml.safe_dump(shop_model_dict_list, f, sort_keys=False)

with open(shop_scenario_file, 'w') as f:
    yaml.safe_dump(shop_scenario_dict_list, f, sort_keys=False)

with open(shop_scenario_set_file, 'w') as f:
    yaml.safe_dump(shop_scenario_set_dict_list, f, sort_keys=False)